<a href="https://colab.research.google.com/github/Pavlos01232/Match_Outcome_Prediction/blob/main/Game_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:

import torch as t
import torch.nn as n
import pandas as pd
import os

print("beep boop")
print("Aston Villa loses")
print("=============================================================")
#GITHUB LOCATION:
#https://github.com/Pavlos01232/Match_Outcome_Prediction
#
#
#
#
#
#
#




#pd.read_csv('https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/PL0304.csv?raw=true')


#"DEEP learning" just means "hidden" layers



#read function
#df = pd.read_csv('https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/PL0405.csv?raw=true',sep='\t', lineterminator='\r')
#print(df)

first = "https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/PL"
last = ".csv?raw=true"
#3 to 23
#the first two files in the training data are formatted incorrectly
#convert csv to dataframe
df=[0]*100000
df2=pd.DataFrame()
i=5
for i in range(5, 23):
  result = first + str('{:02.0f}'.format(i)) + str('{:02.0f}'.format(i+1)) + last
  #print(result)
  x = pd.read_csv(result, sep='\t', encoding = 'unicode_escape', lineterminator='\r')
  df2=pd.concat([df2, x])
  print(len(df2.values))
  #tensor = t.tensor(df.values)
  #print(tensor)
  #df_to_tensor(pd.read_csv(result, sep='\t', encoding = 'unicode_escape', lineterminator='\r'))

print(len(df2.values))
print(df2.values[1][0])
print(df2.values[7][0])
print(df2.values[760][0])
print(df2.values[762][0])
print(df2.values[763][0])
matches=[]
for i in range(1,len(df2.values)):
  #print(df2.values[1][0])
  #print(i)
  matches.append(df2.values[i][0].split(","))


#print (df[5].values[3][0].split(",")[2])
#print (df[5].values[3][0].split(",")[3])
teams=[]
for i in range(1, len(matches)):
  found=False
  for j in range(len(teams)):
    if matches[i][3] == teams[j]:
      found=True
      break
  if found:
    continue
  teams.append(matches[i][3])

print(teams)

print(df[5])
print(df2)
print(len(matches))
print(len(df2))
print(matches[12][2])

#df.to_csv(r'C:\Users\Pavlos\Desktop\export_dataframe.csv', sep='\t', encoding='utf-8')
#print (df[2])
#file_list = os.listdir('https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/')





print("=============================================================")

i=5

stats=t.tensor(i)

#stats[0]=5
#stats[1]=2
#stats[2]=8
#stats[3]=4
#stats[4]=8
#stats[5]=3

print(stats)

t.cuda.memory_allocated(0)/1024**3

beep boop
Aston Villa loses
381


IndexError: index 763 is out of bounds for axis 0 with size 381